In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import itertools
from tensorflow.keras.models import load_model

In [2]:
data = pd.read_csv('hand_landmarks_dataset.csv')

In [3]:
X, y = data.drop(columns=['label','label_id', 'path']), data['label_id']

In [4]:
def super_frame(data, sequence_length = 37, num_features = 63):
    raw_data = data.to_numpy()
    n_sequences = len(raw_data) // sequence_length
    raw_data = raw_data[:n_sequences * sequence_length]
    X_seq = raw_data.reshape((n_sequences, sequence_length, num_features))

    # "Суперкадр" — среднее по последовательности
    X_mean = np.mean(X_seq, axis=1, keepdims=True)  # (n_sequences, 1, 63)

    # Добавляем к последовательности
    X_augmented = np.concatenate([X_seq, X_mean], axis=1)  # (n_sequences, 38, 63)

    X = pd.DataFrame(X_augmented.reshape(38*n_sequences, 63))
    
    return X


In [5]:
prep_X = super_frame(X)

In [6]:
minmax = MinMaxScaler()
X_scaled = minmax.fit_transform(prep_X)

window_size_x = 38
window_size_y = 37
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

X_scaled = X_scaled.reshape(n_samples, 38, 63)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)

In [7]:
y_train = to_categorical(y_train, num_classes=27)
y_test = to_categorical(y_test, num_classes=27)

In [8]:
for i in range(0, 6):
    model = load_model(f'model_{i}_best.h5')
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Model: {i}, Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

Model: 0, Accuracy: 0.4706, Loss: 1.9089


Model: 1, Accuracy: 0.5077, Loss: 1.7983


Model: 2, Accuracy: 0.4255, Loss: 2.0104


Model: 3, Accuracy: 0.5268, Loss: 1.7661


Model: 4, Accuracy: 0.4161, Loss: 2.0242
Model: 5, Accuracy: 0.5451, Loss: 1.7819
